In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from util import *

In [2]:
# from sklearn import set_config
# set_config(display='diagram') 

In [3]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")

testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

subset = list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))

In [6]:
count = training_set.shape[0]

In [7]:
pos_weights = (1 / (training_set[subset].sum() / count).values)

In [8]:
subset = list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))
subset

['AnalysisAndModeling',
 'AnalysisAndModeling_3DModeling',
 'AnatomicalTarget',
 'AnatomicalTarget_LowerExtremity',
 'AnatomicalTarget_LowerExtremity_Hip',
 'AnatomicalTarget_LowerExtremity_Knee',
 'AnatomicalTarget_Torso',
 'AnatomicalTarget_Torso_Spine',
 'AnatomicalTarget_UpperExtremity',
 'AnatomicalTarget_UpperExtremity_Shoulder',
 'Imaging',
 'Imaging_CT',
 'Imaging_MRI',
 'Imaging_Ultrasound',
 'Manufacturing',
 'Manufacturing_AdditiveManufacturing',
 'PersonalizedProduct_Guide/Jig',
 'PersonalizedProduct_Implant',
 'SpecificationofUse',
 'SpecificationofUse_Disease',
 'SpecificationofUse_JointReplacement',
 'SurgicalMethod']

In [9]:
training_labels = training_set[subset]

In [10]:
testing_labels = testing_set[subset]

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [13]:
set(training_set.columns) - set(all_tiers)

{'abstract',
 'all_tiers',
 'all_tiers_100',
 'application_kind',
 'application_number',
 'application_number_formatted',
 'assignees',
 'citations',
 'cited_by',
 'claims',
 'country_code_x',
 'country_code_y',
 'cpc_codes',
 'description',
 'embedding_v1',
 'examiners',
 'family_id',
 'fi_codes',
 'filing_date',
 'fterm_codes',
 'gpa_number',
 'grant_date',
 'inventors',
 'kind',
 'kind_code',
 'padded_serial',
 'pct_number',
 'priority_date',
 'publication_date',
 'publication_number',
 'serial_x',
 'serial_y',
 'similar_npl',
 'similar_patents',
 'tier1_tags',
 'tier1_tier2_tags',
 'tier2_100',
 'tier3_100',
 'title',
 'top_terms',
 'url',
 'uspc_codes'}

In [14]:
training_set.top_terms

0      [prosthesis, member, coupling portion, portion...
1      [femur, tibia, implant, leg, patient, portion,...
2      [lateral, femur, patient, body, medial, slot, ...
3      [glenoid, virtual, alignment pin, patient, sca...
4      [bone, surface, patient, resection, jig, use, ...
                             ...                        
967    [method, bone, medial, patient, template, late...
968    [cutting guide, surgical cutting, customized s...
969    [patient, adapted, surface, surface model, imp...
970    [polymer, monomers, modulus, thiol, multifunct...
971    [implant, patient, peg, portion, bone, porous,...
Name: top_terms, Length: 972, dtype: object

In [15]:
cpc_embeddings = np.fromfile("/home/martin/patentmark/cpc.node2vec.emb.32d.bin", dtype=np.float32).reshape((-1,32))

import joblib
cpc_labelizer = joblib.load('./node2id.joblib')
cpc_lookup = {c: n for n, c in enumerate(cpc_labelizer.classes_)}

@f.collecting
def convert_cpc_codes(codes):
    for code in codes:
        if code in cpc_lookup:
            yield cpc_lookup[code]
    
def embed_cpc_codes(codes):
    embedding = np.zeros(32)
    converted = convert_cpc_codes(codes)
    
    if not converted:
        return embedding
    
    for code_id in converted:
        embedding = embedding + cpc_embeddings[code_id]
        
    return embedding / len(converted)

training_set['embedded_cpc'] = training_set.cpc_codes.apply(embed_cpc_codes)
training_set.embedded_cpc

testing_set['embedded_cpc'] = testing_set.cpc_codes.apply(embed_cpc_codes)
testing_set.embedded_cpc

0      [0.09129103335241477, -0.8074875394503276, -0....
1      [-0.0626441298850945, -0.8264780470303127, -0....
2      [-0.2087969978650411, -0.8326806823412577, -0....
3      [0.020394775830209256, -0.8215901732444764, -0...
4      [-0.26043402403593063, -0.6891247034072876, -0...
                             ...                        
238    [-0.23802674313386282, -0.628900408744812, -0....
239    [-0.3754243354002635, -0.6894144614537557, -0....
240    [-0.12913421913981438, -0.6960149183869362, -0...
241    [-0.3880331997688, -0.702021429171929, -0.2717...
242    [-0.2977850042283535, -0.7015813589096069, -0....
Name: embedded_cpc, Length: 243, dtype: object

In [16]:
np.array(testing_set.embedded_cpc.values.tolist()).shape

(243, 32)

In [17]:
from sklearn.ensemble import *

In [18]:
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [19]:
from sklearn.dummy import DummyClassifier

In [20]:
from sklearn.linear_model import LogisticRegression

In [53]:
# from transformers import FeatureExtractionPipeline, AutoModel, AutoTokenizer
# transformer_model = AutoModel.from_pretrained("bertForPatents/", gradient_checkpointing=True)
# transformer_tokenizer = AutoTokenizer.from_pretrained("bertForPatents/")
# transformer = FeatureExtractionPipeline(transformer_model, tokenizer=transformer_tokenizer, device=0)

import sentence_transformers as st
from sentence_transformers import SentenceTransformer

# model_name = "/home/martin/patentmark/patentmark-charting-app/vectors/sPatent-v2/"
# text_embedder = SentenceTransformer(model_name)
model_name = "bertForPatents/"
word_embedding_model = st.models.Transformer(model_name)
pooling_model = st.models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=True)
text_embedder = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [54]:
class TransformerFeatures(sklearn.base.BaseEstimator):
    def fit(self, X, y):
        return self
        
    def transform(self, X):
        return text_embedder.encode(X)
        

In [55]:
class VectorPassthrough(sklearn.base.BaseEstimator):
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array(X.values.tolist())
        

In [56]:
from sklearn.decomposition import TruncatedSVD

In [57]:

# column_prep = ColumnTransformer([
# #      ('top_terms',
# #       CountVectorizer(analyzer=iden, binary=True, min_df=2),
# #      'top_terms'
# #      ),
#      ('cited_by',
#       Pipeline(steps=[
#           ('cited', CountVectorizer(analyzer=iden, binary=True, min_df=2)),
#           ('cited_svd', TruncatedSVD(n_components=64, random_state=42))]),
#      'cited_by'
#      ),
#     ('inventors',
#      Pipeline(steps=[
#          ('inventors', CountVectorizer(analyzer=iden, binary=True, min_df=2)),
#          ('inventors_svd', TruncatedSVD(n_components=64, random_state=42))]),
#      'inventors'
#      ),
# #      ('citations',
# #        CountVectorizer(analyzer=iden, binary=True, min_df=2),
# #       'citations'
# #       ),
# # #         ('similar_npl',
# # #       CountVectorizer(analyzer=lambda x:x, min_df=2),
# # #      'similar_npl'
# # #      ),
# #         ('similar_patents',
# #       CountVectorizer(analyzer=iden, binary=True, min_df=2),
# #      'similar_patents'
# #      )
# #   ,
# #      ('cpc',
# #       CountVectorizer(analyzer=cpc_split, binary=True, min_df=2),
# #      'cpc_codes'
# #      ),
#     ('embedded_cpc', 
#      VectorPassthrough(),
#      'embedded_cpc'
#     ),
#     ('embedding_v1',
#      VectorPassthrough(),
#      'embedding_v1'
#     ),
#     ('transformer_abstract',
#     TransformerFeatures(),
#      'abstract'
#     ),
#     ('transformer_claims',
#      TransformerFeatures(),
#      'claims'
#     )], verbose=False, n_jobs=1)

# from sklearn.preprocessing import Normalizer
# pipeline = Pipeline(steps=[('columns', column_prep), 
#                            ('norm', Normalizer())])
    

In [58]:
tfidf_default_settings = {
    'lowercase': True, 
    'strip_accents': 'ascii',
    'stop_words' : stopwords,
    'min_df': 5,
    #'max_df': 0.5#,
    #'ngram_range': (1,3)
}

column_prep = ColumnTransformer([
#      ('top_terms',
#       CountVectorizer(analyzer=iden, binary=True, min_df=2),
#      'top_terms'
#      ),
     ('cited_by',
      CountVectorizer(analyzer=iden, binary=True, min_df=2),
     'cited_by'
     ),
    ('inventors',
      CountVectorizer(analyzer=iden, binary=True, min_df=2),
     'inventors'
     ),
#      ('citations',
#        CountVectorizer(analyzer=iden, binary=True, min_df=2),
#       'citations'
#       ),
# #         ('similar_npl',
# #       CountVectorizer(analyzer=lambda x:x, min_df=2),
# #      'similar_npl'
# #      ),
#         ('similar_patents',
#       CountVectorizer(analyzer=iden, binary=True, min_df=2),
#      'similar_patents'
#      )
#   ,
#      ('cpc',
#       CountVectorizer(analyzer=cpc_split, binary=True, min_df=2),
#      'cpc_codes'
#      ),
    ('embedded_cpc', 
     VectorPassthrough(),
     'embedded_cpc'
    ),
    ('embedding_v1',
     VectorPassthrough(),
     'embedding_v1'
    ),
    ('transformer_abstract',
    TransformerFeatures(),
     'abstract'
    ),
    ('transformer_claims',
     TransformerFeatures(),
     'claims'
    )], verbose=False, n_jobs=1)

from sklearn.preprocessing import Normalizer
pipeline = Pipeline(steps=[('columns', column_prep), ('norm', Normalizer())])
    
#     ('abstract_tfidf', 
#     TfidfVectorizer(**tfidf_default_settings),
#    'abstract'),
#     ('claims_tfidf',
#      TfidfVectorizer(**tfidf_default_settings),
#      'claims'
#     ),
#     ('description_tfidf',
#      TfidfVectorizer(**tfidf_default_settings),
#      'description'
#     )
      

In [59]:
from sklearn.decomposition import TruncatedSVD

In [60]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.naive_bayes import GaussianNB

In [61]:
from sklearn.metrics import *

In [62]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
import util
X_train = pipeline.fit_transform(training_set)
X_test = pipeline.transform(testing_set)

In [63]:
y_train = mlb.fit_transform(training_set[subset].apply(util.array_labels, axis=1))
y_test = mlb.transform(testing_set[subset].apply(util.array_labels, axis=1))

In [64]:
def scorer(net, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = (net.predict(ds) >= 0.5)
    #score = sklearn.metrics.label_ranking_loss(y_true, y_pred)
    score = sklearn.metrics.f1_score(y_true, y_pred, average="weighted")
    #score = sklearn.metrics.hamming_loss(y_true, y_pred)
    return score

In [73]:
from torch import nn

import torch.nn.functional as F
def linear_combination(x, y, epsilon): 
    return epsilon*x + (1-epsilon)*y

def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, pos_weights=pos_weights, epsilon:float=0.01, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
        self.pos_weights = pos_weights
    
    def forward(self, preds, target):
        n = preds.size()[-1]
        loss = reduce_loss(-preds.sum(dim=-1), self.reduction)
        nll = F.binary_cross_entropy_with_logits(preds, target, pos_weight=torch.tensor(self.pos_weights).to("cuda:0"), reduction=self.reduction)
        return linear_combination(loss/n, nll, self.epsilon)

In [86]:
from torch import nn
import torch.nn.functional as F
import torch
from skorch import NeuralNet, dataset
from skorch.callbacks import ProgressBar, Checkpoint, EarlyStopping, EpochScoring, LRScheduler, LoadInitState
from skorch.helper import predefined_split

valid_ds = dataset.Dataset(X_test.astype(np.float32), y_test.astype(np.float32))

class Net(nn.Module):
    def __init__(self, input_features, output_features, bottleneck=512):
        super().__init__()
        self.layer1 = nn.Linear(input_features, bottleneck)
        self.dropout = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.layer2 = nn.Linear(bottleneck, bottleneck)
        self.output = nn.Linear(bottleneck, output_features)
        
    def forward(self, x):
        
        x = self.layer1(x)
        x = F.elu(x)
        #x = self.bn(x)
        x = self.dropout(x)
        
        x = self.layer2(x)
        x = F.elu(x)
        x = self.dropout2(x)
        #x = F.elu(x)
        
        x = self.output(x)
        
        return x

checkpoint = Checkpoint(dirname='exp4')
earlystopping = EarlyStopping(patience=20)
#load_state = LoadInitState(checkpoint)

net = NeuralNet(Net, 
                criterion=LabelSmoothingCrossEntropy, 
                lr=0.00005,
                predict_nonlinearity=torch.sigmoid,
                batch_size=32, 
                max_epochs=10000, 
                module__input_features=X_train.shape[1],
                module__output_features=len(mlb.classes_),
                optimizer=torch.optim.AdamW,
                #train_split=predefined_split(valid_ds),
                iterator_train__shuffle=True,
                iterator_train__num_workers=4,
                callbacks=[ProgressBar(),
                           EpochScoring(scorer, name="f1", lower_is_better=False),
                           checkpoint,
                           earlystopping,
                           #load_state,
                           #LRScheduler(monitor='val_loss'),
                          ],
                device='cuda'
               )

In [87]:
X_train.shape

(972, 8572)

In [94]:
net.fit(X_train[:600].astype(np.float32), y_train[:600].astype(np.float32))

Re-initializing module because the following parameters were re-set: input_features, output_features.
Re-initializing optimizer.


  epoch      f1    train_loss    valid_loss    cp     dur
-------  ------  ------------  ------------  ----  ------
      1  0.4312        1.1886        1.1632     +  0.5607


      2  0.5034        1.1830        1.1587     +  0.5797


      3  0.5080        1.1764        1.1534     +  0.5577


      4  0.5080        1.1682        1.1478     +  0.5596


...

    171  0.5265        0.9750        1.0702        0.5577


    172  0.5272        0.9795        1.0685        0.5752


    173  0.5273        0.9731        1.0680        0.5858


Stopping since valid_loss has not improved in the last 20 epochs.


<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=8572, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)

In [95]:
from sklearn import metrics
predictions = (net.predict(X_test.astype(np.float32)) >=0.5).astype(np.long)

print(f"Hamming loss: {metrics.hamming_loss(y_test,predictions)}")
print(metrics.label_ranking_loss(y_test,predictions))
print(metrics.classification_report(y_test, predictions, target_names=mlb.classes_))

print(model_name)
print(pipeline)
print(net)
print(net.get_params())

Hamming loss: 0.4878413767302656
0.6883439099590769
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.36      0.92      0.51        84
           Analysis and Modeling: 3D Modeling       0.31      0.89      0.45        71
                            Anatomical Target       0.69      0.95      0.80       164
           Anatomical Target: Lower Extremity       0.47      0.93      0.62       113
     Anatomical Target: Lower Extremity - Hip       0.21      0.53      0.30        40
    Anatomical Target: Lower Extremity - Knee       0.34      0.93      0.50        82
                     Anatomical Target: Torso       0.15      0.43      0.23        35
             Anatomical Target: Torso - Spine       0.09      0.38      0.14        21
           Anatomical Target: Upper Extremity       0.17      0.48      0.25        31
Anatomical Target: Upper Extremity - Shoulder       0.12      0.39      0.19 

/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
N=250
Hamming loss: 0.7184811073699963
0.9958847736625515

In [ ]:
N=400
Hamming loss: 0.7184811073699963
0.9958847736625515

In [ ]:
N=100
Hamming loss: 0.5875420875420876
0.7901547932844298
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      1.00      0.51        84
           Analysis and Modeling: 3D Modeling       0.29      1.00      0.45        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.00      0.00      0.00        40
    Anatomical Target: Lower Extremity - Knee       0.34      1.00      0.50        82
                     Anatomical Target: Torso       0.00      0.00      0.00        35
             Anatomical Target: Torso - Spine       0.00      0.00      0.00        21
           Anatomical Target: Upper Extremity       0.13      1.00      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.09      1.00      0.17        23
                                      Imaging       0.55      1.00      0.71       133
                                  Imaging: CT       0.24      1.00      0.39        59
                                 Imaging: MRI       0.24      1.00      0.39        59
                          Imaging: Ultrasound       0.13      1.00      0.23        32
                                Manufacturing       0.34      1.00      0.51        83
        Manufacturing: Additive Manufacturing       0.16      1.00      0.27        38
           Personalized Product: Guide or Jig       0.49      1.00      0.66       120
                Personalized Product: Implant       0.51      1.00      0.68       124
                         Specification of Use       0.33      1.00      0.49        79
                Specification of Use: Disease       0.12      1.00      0.22        30
      Specification of Use: Joint Replacement       0.18      1.00      0.31        44
                              Surgical Method       0.00      0.00      0.00        40

                                    micro avg       0.31      0.91      0.47      1505
                                    macro avg       0.26      0.82      0.37      1505
                                 weighted avg       0.36      0.91      0.50      1505
                                  samples avg       0.31      0.91      0.45      1505

bertForPatents/
Pipeline(steps=[('columns',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('cited_by',
                                                  CountVectorizer(analyzer=<function iden at 0x7fbdbdddc430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'cited_by'),
                                                 ('inventors',
                                                  CountVectorizer(analyzer=<function iden at 0x7fbdbdddc430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'inventors'),
                                                 ('embedded_cpc',
                                                  VectorPassthrough(),
                                                  'embedded_cpc'),
                                                 ('embedding_v1',
                                                  VectorPassthrough(),
                                                  'embedding_v1'),
                                                 ('transformer_abstract',
                                                  TransformerFeatures(),
                                                  'abstract'),
                                                 ('transformer_claims',
                                                  TransformerFeatures(),
                                                  'claims')])),
                ('norm', Normalizer())])
<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=8572, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)
{'module': <class '__main__.Net'>, 'criterion': <class '__main__.LabelSmoothingCrossEntropy'>, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 5e-05, 'max_epochs': 10000, 'batch_size': 32, 'iterator_train': <class 'torch.utils.data.dataloader.DataLoader'>, 'iterator_valid': <class 'torch.utils.data.dataloader.DataLoader'>, 'dataset': <class 'skorch.dataset.Dataset'>, 'train_split': <skorch.dataset.CVSplit object at 0x7fbc5f11cdc0>, 'callbacks': [<skorch.callbacks.logging.ProgressBar object at 0x7fbbc112ae50>, <skorch.callbacks.scoring.EpochScoring object at 0x7fbbc111da30>, <skorch.callbacks.training.Checkpoint object at 0x7fbc93feab20>, <skorch.callbacks.training.EarlyStopping object at 0x7fbba17d26a0>], 'predict_nonlinearity': <built-in method sigmoid of type object at 0x7fbc8d3b55a0>, 'warm_start': False, 'verbose': 1, 'device': 'cuda', 'module__input_features': 8572, 'module__output_features': 22, 'iterator_train__shuffle': True, 'iterator_train__num_workers': 4, 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer object at 0x7fbba164fb50>, 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fbba13fca30>, 'callbacks__train_loss__name': 'train_loss', 'callbacks__train_loss__lower_is_better': True, 'callbacks__train_loss__on_train': True, 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fbba13fce50>, 'callbacks__valid_loss__name': 'valid_loss', 'callbacks__valid_loss__lower_is_better': True, 'callbacks__valid_loss__on_train': False, 'callbacks__ProgressBar': <skorch.callbacks.logging.ProgressBar object at 0x7fbbc112ae50>, 'callbacks__ProgressBar__batches_per_epoch': 'auto', 'callbacks__ProgressBar__detect_notebook': True, 'callbacks__ProgressBar__postfix_keys': ['train_loss', 'valid_loss'], 'callbacks__EpochScoring': <skorch.callbacks.scoring.EpochScoring object at 0x7fbbc111da30>, 'callbacks__EpochScoring__scoring': <function scorer at 0x7fbc5f07e160>, 'callbacks__EpochScoring__lower_is_better': False, 'callbacks__EpochScoring__on_train': False, 'callbacks__EpochScoring__name': 'f1', 'callbacks__EpochScoring__target_extractor': <function to_numpy at 0x7fbc5f1219d0>, 'callbacks__EpochScoring__use_caching': True, 'callbacks__Checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fbc93feab20>, 'callbacks__Checkpoint__monitor': 'valid_loss_best', 'callbacks__Checkpoint__f_params': 'params.pt', 'callbacks__Checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__Checkpoint__f_criterion': 'criterion.pt', 'callbacks__Checkpoint__f_history': 'history.json', 'callbacks__Checkpoint__f_pickle': None, 'callbacks__Checkpoint__fn_prefix': '', 'callbacks__Checkpoint__dirname': 'exp4', 'callbacks__Checkpoint__event_name': 'event_cp', 'callbacks__Checkpoint__sink': <function noop at 0x7fbc5f1203a0>, 'callbacks__EarlyStopping': <skorch.callbacks.training.EarlyStopping object at 0x7fbba17d26a0>, 'callbacks__EarlyStopping__monitor': 'valid_loss', 'callbacks__EarlyStopping__lower_is_better': True, 'callbacks__EarlyStopping__patience': 20, 'callbacks__EarlyStopping__threshold': 0.0001, 'callbacks__EarlyStopping__threshold_mode': 'rel', 'callbacks__EarlyStopping__sink': <built-in function print>, 'callbacks__print_log': <skorch.callbacks.logging.PrintLog object at 0x7fbba13fc2e0>, 'callbacks__print_log__keys_ignored': None, 'callbacks__print_log__sink': <built-in function print>, 'callbacks__print_log__tablefmt': 'simple', 'callbacks__print_log__floatfmt': '.4f', 'callbacks__print_log__stralign': 'right'}
/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

In [ ]:
Hamming loss: 0.5044893378226711
0.6950069237607938
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.36      0.93      0.52        84
           Analysis and Modeling: 3D Modeling       0.31      0.93      0.47        71
                            Anatomical Target       0.68      0.95      0.80       164
           Anatomical Target: Lower Extremity       0.48      0.96      0.64       113
     Anatomical Target: Lower Extremity - Hip       0.20      0.75      0.32        40
    Anatomical Target: Lower Extremity - Knee       0.35      0.91      0.51        82
                     Anatomical Target: Torso       0.14      0.57      0.23        35
             Anatomical Target: Torso - Spine       0.08      0.38      0.13        21
           Anatomical Target: Upper Extremity       0.17      0.58      0.26        31
Anatomical Target: Upper Extremity - Shoulder       0.11      0.43      0.18        23
                                      Imaging       0.57      0.98      0.72       133
                                  Imaging: CT       0.28      0.90      0.43        59
                                 Imaging: MRI       0.28      0.88      0.42        59
                          Imaging: Ultrasound       0.13      0.31      0.18        32
                                Manufacturing       0.35      0.90      0.51        83
        Manufacturing: Additive Manufacturing       0.18      0.71      0.29        38
           Personalized Product: Guide or Jig       0.49      0.90      0.64       120
                Personalized Product: Implant       0.52      0.98      0.68       124
                         Specification of Use       0.33      0.95      0.49        79
                Specification of Use: Disease       0.12      0.30      0.17        30
      Specification of Use: Joint Replacement       0.19      0.95      0.31        44
                              Surgical Method       0.23      0.45      0.31        40

                                    micro avg       0.34      0.86      0.49      1505
                                    macro avg       0.30      0.76      0.42      1505
                                 weighted avg       0.39      0.86      0.53      1505
                                  samples avg       0.36      0.86      0.48      1505

bertForPatents/
Pipeline(steps=[('columns',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('cited_by',
                                                  CountVectorizer(analyzer=<function iden at 0x7fbdbdddc430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'cited_by'),
                                                 ('inventors',
                                                  CountVectorizer(analyzer=<function iden at 0x7fbdbdddc430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'inventors'),
                                                 ('embedded_cpc',
                                                  VectorPassthrough(),
                                                  'embedded_cpc'),
                                                 ('embedding_v1',
                                                  VectorPassthrough(),
                                                  'embedding_v1'),
                                                 ('transformer_abstract',
                                                  TransformerFeatures(),
                                                  'abstract'),
                                                 ('transformer_claims',
                                                  TransformerFeatures(),
                                                  'claims')])),
                ('norm', Normalizer())])
<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=8572, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)
{'module': <class '__main__.Net'>, 'criterion': <class '__main__.LabelSmoothingCrossEntropy'>, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 5e-05, 'max_epochs': 10000, 'batch_size': 32, 'iterator_train': <class 'torch.utils.data.dataloader.DataLoader'>, 'iterator_valid': <class 'torch.utils.data.dataloader.DataLoader'>, 'dataset': <class 'skorch.dataset.Dataset'>, 'train_split': <skorch.dataset.CVSplit object at 0x7fbc5f11cdc0>, 'callbacks': [<skorch.callbacks.logging.ProgressBar object at 0x7fbba0ef6a60>, <skorch.callbacks.scoring.EpochScoring object at 0x7fbbc09913d0>, <skorch.callbacks.training.Checkpoint object at 0x7fbbc1354fd0>, <skorch.callbacks.training.EarlyStopping object at 0x7fbbc09910a0>], 'predict_nonlinearity': <built-in method sigmoid of type object at 0x7fbc8d3b55a0>, 'warm_start': False, 'verbose': 1, 'device': 'cuda', 'module__input_features': 8572, 'module__output_features': 22, 'iterator_train__shuffle': True, 'iterator_train__num_workers': 4, 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer object at 0x7fbba16d6730>, 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fbbc12b8370>, 'callbacks__train_loss__name': 'train_loss', 'callbacks__train_loss__lower_is_better': True, 'callbacks__train_loss__on_train': True, 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fbba163df40>, 'callbacks__valid_loss__name': 'valid_loss', 'callbacks__valid_loss__lower_is_better': True, 'callbacks__valid_loss__on_train': False, 'callbacks__ProgressBar': <skorch.callbacks.logging.ProgressBar object at 0x7fbba0ef6a60>, 'callbacks__ProgressBar__batches_per_epoch': 'auto', 'callbacks__ProgressBar__detect_notebook': True, 'callbacks__ProgressBar__postfix_keys': ['train_loss', 'valid_loss'], 'callbacks__EpochScoring': <skorch.callbacks.scoring.EpochScoring object at 0x7fbbc09913d0>, 'callbacks__EpochScoring__scoring': <function scorer at 0x7fbc5f07e160>, 'callbacks__EpochScoring__lower_is_better': False, 'callbacks__EpochScoring__on_train': False, 'callbacks__EpochScoring__name': 'f1', 'callbacks__EpochScoring__target_extractor': <function to_numpy at 0x7fbc5f1219d0>, 'callbacks__EpochScoring__use_caching': True, 'callbacks__Checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fbbc1354fd0>, 'callbacks__Checkpoint__monitor': 'valid_loss_best', 'callbacks__Checkpoint__f_params': 'params.pt', 'callbacks__Checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__Checkpoint__f_criterion': 'criterion.pt', 'callbacks__Checkpoint__f_history': 'history.json', 'callbacks__Checkpoint__f_pickle': None, 'callbacks__Checkpoint__fn_prefix': '', 'callbacks__Checkpoint__dirname': 'exp3', 'callbacks__Checkpoint__event_name': 'event_cp', 'callbacks__Checkpoint__sink': <function noop at 0x7fbc5f1203a0>, 'callbacks__EarlyStopping': <skorch.callbacks.training.EarlyStopping object at 0x7fbbc09910a0>, 'callbacks__EarlyStopping__monitor': 'valid_loss', 'callbacks__EarlyStopping__lower_is_better': True, 'callbacks__EarlyStopping__patience': 20, 'callbacks__EarlyStopping__threshold': 0.0001, 'callbacks__EarlyStopping__threshold_mode': 'rel', 'callbacks__EarlyStopping__sink': <built-in function print>, 'callbacks__print_log': <skorch.callbacks.logging.PrintLog object at 0x7fbba163ddf0>, 'callbacks__print_log__keys_ignored': None, 'callbacks__print_log__sink': <built-in function print>, 'callbacks__print_log__tablefmt': 'simple', 'callbacks__print_log__floatfmt': '.4f', 'callbacks__print_log__stralign': 'right'}

In [ ]:
Best so far

Hamming loss: 0.5422745978301534
0.7398683245697636
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      0.96      0.52        84
           Analysis and Modeling: 3D Modeling       0.30      0.94      0.46        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.64       113
     Anatomical Target: Lower Extremity - Hip       0.18      0.78      0.30        40
    Anatomical Target: Lower Extremity - Knee       0.34      0.96      0.51        82
                     Anatomical Target: Torso       0.14      0.63      0.24        35
             Anatomical Target: Torso - Spine       0.09      0.43      0.15        21
           Anatomical Target: Upper Extremity       0.14      0.58      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.12      0.57      0.20        23
                                      Imaging       0.56      0.99      0.72       133
                                  Imaging: CT       0.26      0.97      0.41        59
                                 Imaging: MRI       0.26      0.95      0.41        59
                          Imaging: Ultrasound       0.16      0.59      0.25        32
                                Manufacturing       0.35      0.93      0.51        83
        Manufacturing: Additive Manufacturing       0.21      0.79      0.33        38
           Personalized Product: Guide or Jig       0.50      0.98      0.66       120
                Personalized Product: Implant       0.52      0.99      0.68       124
                         Specification of Use       0.34      0.97      0.51        79
                Specification of Use: Disease       0.13      0.50      0.20        30
      Specification of Use: Joint Replacement       0.18      0.93      0.30        44
                              Surgical Method       0.23      0.68      0.35        40

                                    micro avg       0.33      0.91      0.49      1505
                                    macro avg       0.30      0.82      0.43      1505
                                 weighted avg       0.39      0.91      0.53      1505
                                  samples avg       0.34      0.92      0.47      1505

distilroberta-base-msmarco-v2
Pipeline(steps=[('columns',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('cited_by',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'cited_by'),
                                                 ('inventors',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'inventors'),
                                                 ('embedded_cpc',
                                                  VectorPassthrough(),
                                                  'embedded_cpc'),
                                                 ('embedding_v1',
                                                  VectorPassthrough(),
                                                  'embedding_v1'),
                                                 ('transformer_abstract',
                                                  TransformerFeatures(),
                                                  'abstract'),
                                                 ('transformer_claims',
                                                  TransformerFeatures(),
                                                  'claims')])),
                ('norm', Normalizer())])
<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=6012, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)
{'module': <class '__main__.Net'>, 'criterion': <class '__main__.LabelSmoothingCrossEntropy'>, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'max_epochs': 10000, 'batch_size': 32, 'iterator_train': <class 'torch.utils.data.dataloader.DataLoader'>, 'iterator_valid': <class 'torch.utils.data.dataloader.DataLoader'>, 'dataset': <class 'skorch.dataset.Dataset'>, 'train_split': <skorch.dataset.CVSplit object at 0x7fc7d033f460>, 'callbacks': [<skorch.callbacks.logging.ProgressBar object at 0x7fc7d0381100>, <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d0304370>, <skorch.callbacks.training.Checkpoint object at 0x7fc7d0381040>, <skorch.callbacks.training.EarlyStopping object at 0x7fc7d0381130>, <skorch.callbacks.training.LoadInitState object at 0x7fc7d03810d0>], 'predict_nonlinearity': <built-in method sigmoid of type object at 0x7fc88c1305a0>, 'warm_start': False, 'verbose': 1, 'device': 'cuda', 'module__input_features': 6012, 'module__output_features': 22, 'iterator_train__shuffle': True, 'iterator_train__num_workers': 4, 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer object at 0x7fc7c0e793d0>, 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc7c1e4fa60>, 'callbacks__train_loss__name': 'train_loss', 'callbacks__train_loss__lower_is_better': True, 'callbacks__train_loss__on_train': True, 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc7c1e4f190>, 'callbacks__valid_loss__name': 'valid_loss', 'callbacks__valid_loss__lower_is_better': True, 'callbacks__valid_loss__on_train': False, 'callbacks__ProgressBar': <skorch.callbacks.logging.ProgressBar object at 0x7fc7d0381100>, 'callbacks__ProgressBar__batches_per_epoch': 'auto', 'callbacks__ProgressBar__detect_notebook': True, 'callbacks__ProgressBar__postfix_keys': ['train_loss', 'valid_loss'], 'callbacks__EpochScoring': <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d0304370>, 'callbacks__EpochScoring__scoring': <function scorer at 0x7fc7d172ef70>, 'callbacks__EpochScoring__lower_is_better': False, 'callbacks__EpochScoring__on_train': False, 'callbacks__EpochScoring__name': 'f1', 'callbacks__EpochScoring__target_extractor': <function to_numpy at 0x7fc7d0383550>, 'callbacks__EpochScoring__use_caching': True, 'callbacks__Checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc7d0381040>, 'callbacks__Checkpoint__monitor': 'valid_loss_best', 'callbacks__Checkpoint__f_params': 'params.pt', 'callbacks__Checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__Checkpoint__f_criterion': 'criterion.pt', 'callbacks__Checkpoint__f_history': 'history.json', 'callbacks__Checkpoint__f_pickle': None, 'callbacks__Checkpoint__fn_prefix': '', 'callbacks__Checkpoint__dirname': 'exp2', 'callbacks__Checkpoint__event_name': 'event_cp', 'callbacks__Checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__EarlyStopping': <skorch.callbacks.training.EarlyStopping object at 0x7fc7d0381130>, 'callbacks__EarlyStopping__monitor': 'valid_loss', 'callbacks__EarlyStopping__lower_is_better': True, 'callbacks__EarlyStopping__patience': 20, 'callbacks__EarlyStopping__threshold': 0.0001, 'callbacks__EarlyStopping__threshold_mode': 'rel', 'callbacks__EarlyStopping__sink': <built-in function print>, 'callbacks__LoadInitState': <skorch.callbacks.training.LoadInitState object at 0x7fc7d03810d0>, 'callbacks__LoadInitState__checkpoint__monitor': 'valid_loss_best', 'callbacks__LoadInitState__checkpoint__f_params': 'params.pt', 'callbacks__LoadInitState__checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__LoadInitState__checkpoint__f_criterion': 'criterion.pt', 'callbacks__LoadInitState__checkpoint__f_history': 'history.json', 'callbacks__LoadInitState__checkpoint__f_pickle': None, 'callbacks__LoadInitState__checkpoint__fn_prefix': '', 'callbacks__LoadInitState__checkpoint__dirname': 'exp2', 'callbacks__LoadInitState__checkpoint__event_name': 'event_cp', 'callbacks__LoadInitState__checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__LoadInitState__checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc7d0381040>, 'callbacks__print_log': <skorch.callbacks.logging.PrintLog object at 0x7fc7c1e4f460>, 'callbacks__print_log__keys_ignored': None, 'callbacks__print_log__sink': <built-in function print>, 'callbacks__print_log__tablefmt': 'simple', 'callbacks__print_log__floatfmt': '.4f', 'callbacks__print_log__stralign': 'right'}
/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

In [ ]:
Hamming loss: 0.5744481855592967
0.7836308184196901
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.36      0.96      0.52        84
           Analysis and Modeling: 3D Modeling       0.30      0.96      0.46        71
                            Anatomical Target       0.68      0.99      0.80       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.19      0.85      0.31        40
    Anatomical Target: Lower Extremity - Knee       0.33      0.94      0.49        82
                     Anatomical Target: Torso       0.15      0.77      0.25        35
             Anatomical Target: Torso - Spine       0.09      0.67      0.15        21
           Anatomical Target: Upper Extremity       0.15      0.81      0.25        31
Anatomical Target: Upper Extremity - Shoulder       0.11      0.78      0.19        23
                                      Imaging       0.56      0.98      0.71       133
                                  Imaging: CT       0.26      0.98      0.41        59
                                 Imaging: MRI       0.25      0.95      0.39        59
                          Imaging: Ultrasound       0.15      0.47      0.23        32
                                Manufacturing       0.35      0.93      0.50        83
        Manufacturing: Additive Manufacturing       0.17      0.87      0.28        38
           Personalized Product: Guide or Jig       0.51      0.96      0.66       120
                Personalized Product: Implant       0.52      0.98      0.68       124
                         Specification of Use       0.33      0.96      0.49        79
                Specification of Use: Disease       0.13      0.33      0.18        30
      Specification of Use: Joint Replacement       0.19      0.98      0.32        44
                              Surgical Method       0.23      0.53      0.32        40

                                    micro avg       0.32      0.91      0.47      1505
                                    macro avg       0.29      0.85      0.42      1505
                                 weighted avg       0.39      0.91      0.53      1505
                                  samples avg       0.34      0.92      0.46      1505

/home/martin/patentmark/patentmark-charting-app/vectors/patent-electra-v4-patent-mnr-cl/
Pipeline(steps=[('columns',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('cited_by',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'cited_by'),
                                                 ('inventors',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'inventors'),
                                                 ('embedded_cpc',
                                                  VectorPassthrough(),
                                                  'embedded_cpc'),
                                                 ('embedding_v1',
                                                  VectorPassthrough(),
                                                  'embedding_v1'),
                                                 ('transformer_abstract',
                                                  TransformerFeatures(),
                                                  'abstract'),
                                                 ('transformer_claims',
                                                  TransformerFeatures(),
                                                  'claims')])),
                ('norm', Normalizer())])
<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=6012, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)
{'module': <class '__main__.Net'>, 'criterion': <class '__main__.LabelSmoothingCrossEntropy'>, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 5e-05, 'max_epochs': 10000, 'batch_size': 32, 'iterator_train': <class 'torch.utils.data.dataloader.DataLoader'>, 'iterator_valid': <class 'torch.utils.data.dataloader.DataLoader'>, 'dataset': <class 'skorch.dataset.Dataset'>, 'train_split': <skorch.dataset.CVSplit object at 0x7fc7d033f460>, 'callbacks': [<skorch.callbacks.logging.ProgressBar object at 0x7fc7d16d18e0>, <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d16d1610>, <skorch.callbacks.training.Checkpoint object at 0x7fc874db38e0>, <skorch.callbacks.training.EarlyStopping object at 0x7fc7d16d17f0>, <skorch.callbacks.training.LoadInitState object at 0x7fc88ca963a0>], 'predict_nonlinearity': <built-in method sigmoid of type object at 0x7fc88c1305a0>, 'warm_start': False, 'verbose': 1, 'device': 'cuda', 'module__input_features': 6012, 'module__output_features': 22, 'iterator_train__shuffle': True, 'iterator_train__num_workers': 4, 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer object at 0x7fc874d5e2b0>, 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc79ee405b0>, 'callbacks__train_loss__name': 'train_loss', 'callbacks__train_loss__lower_is_better': True, 'callbacks__train_loss__on_train': True, 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc874db3d30>, 'callbacks__valid_loss__name': 'valid_loss', 'callbacks__valid_loss__lower_is_better': True, 'callbacks__valid_loss__on_train': False, 'callbacks__ProgressBar': <skorch.callbacks.logging.ProgressBar object at 0x7fc7d16d18e0>, 'callbacks__ProgressBar__batches_per_epoch': 'auto', 'callbacks__ProgressBar__detect_notebook': True, 'callbacks__ProgressBar__postfix_keys': ['train_loss', 'valid_loss'], 'callbacks__EpochScoring': <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d16d1610>, 'callbacks__EpochScoring__scoring': <function scorer at 0x7fc85dcba310>, 'callbacks__EpochScoring__lower_is_better': False, 'callbacks__EpochScoring__on_train': False, 'callbacks__EpochScoring__name': 'f1', 'callbacks__EpochScoring__target_extractor': <function to_numpy at 0x7fc7d0383550>, 'callbacks__EpochScoring__use_caching': True, 'callbacks__Checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc874db38e0>, 'callbacks__Checkpoint__monitor': 'valid_loss_best', 'callbacks__Checkpoint__f_params': 'params.pt', 'callbacks__Checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__Checkpoint__f_criterion': 'criterion.pt', 'callbacks__Checkpoint__f_history': 'history.json', 'callbacks__Checkpoint__f_pickle': None, 'callbacks__Checkpoint__fn_prefix': '', 'callbacks__Checkpoint__dirname': 'exp2', 'callbacks__Checkpoint__event_name': 'event_cp', 'callbacks__Checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__EarlyStopping': <skorch.callbacks.training.EarlyStopping object at 0x7fc7d16d17f0>, 'callbacks__EarlyStopping__monitor': 'valid_loss', 'callbacks__EarlyStopping__lower_is_better': True, 'callbacks__EarlyStopping__patience': 20, 'callbacks__EarlyStopping__threshold': 0.0001, 'callbacks__EarlyStopping__threshold_mode': 'rel', 'callbacks__EarlyStopping__sink': <built-in function print>, 'callbacks__LoadInitState': <skorch.callbacks.training.LoadInitState object at 0x7fc88ca963a0>, 'callbacks__LoadInitState__checkpoint__monitor': 'valid_loss_best', 'callbacks__LoadInitState__checkpoint__f_params': 'params.pt', 'callbacks__LoadInitState__checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__LoadInitState__checkpoint__f_criterion': 'criterion.pt', 'callbacks__LoadInitState__checkpoint__f_history': 'history.json', 'callbacks__LoadInitState__checkpoint__f_pickle': None, 'callbacks__LoadInitState__checkpoint__fn_prefix': '', 'callbacks__LoadInitState__checkpoint__dirname': 'exp2', 'callbacks__LoadInitState__checkpoint__event_name': 'event_cp', 'callbacks__LoadInitState__checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__LoadInitState__checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc874db38e0>, 'callbacks__print_log': <skorch.callbacks.logging.PrintLog object at 0x7fc7d03678b0>, 'callbacks__print_log__keys_ignored': None, 'callbacks__print_log__sink': <built-in function print>, 'callbacks__print_log__tablefmt': 'simple', 'callbacks__print_log__floatfmt': '.4f', 'callbacks__print_log__stralign': 'right'}
/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

In [ ]:


Hamming loss: 0.522633744855967
0.7151155295024978
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.36      0.94      0.52        84
           Analysis and Modeling: 3D Modeling       0.29      0.89      0.44        71
                            Anatomical Target       0.68      0.96      0.80       164
           Anatomical Target: Lower Extremity       0.48      0.97      0.65       113
     Anatomical Target: Lower Extremity - Hip       0.19      0.82      0.31        40
    Anatomical Target: Lower Extremity - Knee       0.35      0.91      0.50        82
                     Anatomical Target: Torso       0.16      0.69      0.26        35
             Anatomical Target: Torso - Spine       0.08      0.48      0.14        21
           Anatomical Target: Upper Extremity       0.16      0.68      0.25        31
Anatomical Target: Upper Extremity - Shoulder       0.12      0.61      0.20        23
                                      Imaging       0.56      0.98      0.71       133
                                  Imaging: CT       0.28      0.97      0.43        59
                                 Imaging: MRI       0.27      0.93      0.41        59
                          Imaging: Ultrasound       0.11      0.31      0.17        32
                                Manufacturing       0.35      0.92      0.51        83
        Manufacturing: Additive Manufacturing       0.19      0.74      0.31        38
           Personalized Product: Guide or Jig       0.51      0.93      0.66       120
                Personalized Product: Implant       0.52      0.99      0.68       124
                         Specification of Use       0.33      0.95      0.49        79
                Specification of Use: Disease       0.14      0.37      0.20        30
      Specification of Use: Joint Replacement       0.19      0.91      0.31        44
                              Surgical Method       0.27      0.55      0.36        40

                                    micro avg       0.34      0.88      0.49      1505
                                    macro avg       0.30      0.79      0.42      1505
                                 weighted avg       0.39      0.88      0.53      1505
                                  samples avg       0.35      0.89      0.48      1505
bertForPatents/
Pipeline(steps=[('columns',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('cited_by',
                                                  CountVectorizer(analyzer=<function iden at 0x7fd77b563d30>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'cited_by'),
                                                 ('inventors',
                                                  CountVectorizer(analyzer=<function iden at 0x7fd77b563d30>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'inventors'),
                                                 ('embedded_cpc',
                                                  VectorPassthrough(),
                                                  'embedded_cpc'),
                                                 ('embedding_v1',
                                                  VectorPassthrough(),
                                                  'embedding_v1'),
                                                 ('transformer_abstract',
                                                  TransformerFeatures(),
                                                  'abstract'),
                                                 ('transformer_claims',
                                                  TransformerFeatures(),
                                                  'claims')])),
                ('norm', Normalizer())])
<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=8572, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)


In [ ]:
Hamming loss: 0.5744481855592967
0.7836308184196901
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.36      0.96      0.52        84
           Analysis and Modeling: 3D Modeling       0.30      0.96      0.46        71
                            Anatomical Target       0.68      0.99      0.80       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.19      0.85      0.31        40
    Anatomical Target: Lower Extremity - Knee       0.33      0.94      0.49        82
                     Anatomical Target: Torso       0.15      0.77      0.25        35
             Anatomical Target: Torso - Spine       0.09      0.67      0.15        21
           Anatomical Target: Upper Extremity       0.15      0.81      0.25        31
Anatomical Target: Upper Extremity - Shoulder       0.11      0.78      0.19        23
                                      Imaging       0.56      0.98      0.71       133
                                  Imaging: CT       0.26      0.98      0.41        59
                                 Imaging: MRI       0.25      0.95      0.39        59
                          Imaging: Ultrasound       0.15      0.47      0.23        32
                                Manufacturing       0.35      0.93      0.50        83
        Manufacturing: Additive Manufacturing       0.17      0.87      0.28        38
           Personalized Product: Guide or Jig       0.51      0.96      0.66       120
                Personalized Product: Implant       0.52      0.98      0.68       124
                         Specification of Use       0.33      0.96      0.49        79
                Specification of Use: Disease       0.13      0.33      0.18        30
      Specification of Use: Joint Replacement       0.19      0.98      0.32        44
                              Surgical Method       0.23      0.53      0.32        40

                                    micro avg       0.32      0.91      0.47      1505
                                    macro avg       0.29      0.85      0.42      1505
                                 weighted avg       0.39      0.91      0.53      1505
                                  samples avg       0.34      0.92      0.46      1505

/home/martin/patentmark/patentmark-charting-app/vectors/patent-electra-v4-patent-mnr-cl/
Pipeline(steps=[('columns',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('cited_by',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'cited_by'),
                                                 ('inventors',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'inventors'),
                                                 ('embedded_cpc',
                                                  VectorPassthrough(),
                                                  'embedded_cpc'),
                                                 ('embedding_v1',
                                                  VectorPassthrough(),
                                                  'embedding_v1'),
                                                 ('transformer_abstract',
                                                  TransformerFeatures(),
                                                  'abstract'),
                                                 ('transformer_claims',
                                                  TransformerFeatures(),
                                                  'claims')])),
                ('norm', Normalizer())])
<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=6012, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)
{'module': <class '__main__.Net'>, 'criterion': <class '__main__.LabelSmoothingCrossEntropy'>, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 5e-05, 'max_epochs': 10000, 'batch_size': 32, 'iterator_train': <class 'torch.utils.data.dataloader.DataLoader'>, 'iterator_valid': <class 'torch.utils.data.dataloader.DataLoader'>, 'dataset': <class 'skorch.dataset.Dataset'>, 'train_split': <skorch.dataset.CVSplit object at 0x7fc7d033f460>, 'callbacks': [<skorch.callbacks.logging.ProgressBar object at 0x7fc7d16d18e0>, <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d16d1610>, <skorch.callbacks.training.Checkpoint object at 0x7fc874db38e0>, <skorch.callbacks.training.EarlyStopping object at 0x7fc7d16d17f0>, <skorch.callbacks.training.LoadInitState object at 0x7fc88ca963a0>], 'predict_nonlinearity': <built-in method sigmoid of type object at 0x7fc88c1305a0>, 'warm_start': False, 'verbose': 1, 'device': 'cuda', 'module__input_features': 6012, 'module__output_features': 22, 'iterator_train__shuffle': True, 'iterator_train__num_workers': 4, 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer object at 0x7fc874d5e2b0>, 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc79ee405b0>, 'callbacks__train_loss__name': 'train_loss', 'callbacks__train_loss__lower_is_better': True, 'callbacks__train_loss__on_train': True, 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc874db3d30>, 'callbacks__valid_loss__name': 'valid_loss', 'callbacks__valid_loss__lower_is_better': True, 'callbacks__valid_loss__on_train': False, 'callbacks__ProgressBar': <skorch.callbacks.logging.ProgressBar object at 0x7fc7d16d18e0>, 'callbacks__ProgressBar__batches_per_epoch': 'auto', 'callbacks__ProgressBar__detect_notebook': True, 'callbacks__ProgressBar__postfix_keys': ['train_loss', 'valid_loss'], 'callbacks__EpochScoring': <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d16d1610>, 'callbacks__EpochScoring__scoring': <function scorer at 0x7fc85dcba310>, 'callbacks__EpochScoring__lower_is_better': False, 'callbacks__EpochScoring__on_train': False, 'callbacks__EpochScoring__name': 'f1', 'callbacks__EpochScoring__target_extractor': <function to_numpy at 0x7fc7d0383550>, 'callbacks__EpochScoring__use_caching': True, 'callbacks__Checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc874db38e0>, 'callbacks__Checkpoint__monitor': 'valid_loss_best', 'callbacks__Checkpoint__f_params': 'params.pt', 'callbacks__Checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__Checkpoint__f_criterion': 'criterion.pt', 'callbacks__Checkpoint__f_history': 'history.json', 'callbacks__Checkpoint__f_pickle': None, 'callbacks__Checkpoint__fn_prefix': '', 'callbacks__Checkpoint__dirname': 'exp2', 'callbacks__Checkpoint__event_name': 'event_cp', 'callbacks__Checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__EarlyStopping': <skorch.callbacks.training.EarlyStopping object at 0x7fc7d16d17f0>, 'callbacks__EarlyStopping__monitor': 'valid_loss', 'callbacks__EarlyStopping__lower_is_better': True, 'callbacks__EarlyStopping__patience': 20, 'callbacks__EarlyStopping__threshold': 0.0001, 'callbacks__EarlyStopping__threshold_mode': 'rel', 'callbacks__EarlyStopping__sink': <built-in function print>, 'callbacks__LoadInitState': <skorch.callbacks.training.LoadInitState object at 0x7fc88ca963a0>, 'callbacks__LoadInitState__checkpoint__monitor': 'valid_loss_best', 'callbacks__LoadInitState__checkpoint__f_params': 'params.pt', 'callbacks__LoadInitState__checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__LoadInitState__checkpoint__f_criterion': 'criterion.pt', 'callbacks__LoadInitState__checkpoint__f_history': 'history.json', 'callbacks__LoadInitState__checkpoint__f_pickle': None, 'callbacks__LoadInitState__checkpoint__fn_prefix': '', 'callbacks__LoadInitState__checkpoint__dirname': 'exp2', 'callbacks__LoadInitState__checkpoint__event_name': 'event_cp', 'callbacks__LoadInitState__checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__LoadInitState__checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc874db38e0>, 'callbacks__print_log': <skorch.callbacks.logging.PrintLog object at 0x7fc7d03678b0>, 'callbacks__print_log__keys_ignored': None, 'callbacks__print_log__sink': <built-in function print>, 'callbacks__print_log__tablefmt': 'simple', 'callbacks__print_log__floatfmt': '.4f', 'callbacks__print_log__stralign': 'right'}

In [ ]:
Hamming loss: 0.5422745978301534
0.7398683245697636
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      0.96      0.52        84
           Analysis and Modeling: 3D Modeling       0.30      0.94      0.46        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.64       113
     Anatomical Target: Lower Extremity - Hip       0.18      0.78      0.30        40
    Anatomical Target: Lower Extremity - Knee       0.34      0.96      0.51        82
                     Anatomical Target: Torso       0.14      0.63      0.24        35
             Anatomical Target: Torso - Spine       0.09      0.43      0.15        21
           Anatomical Target: Upper Extremity       0.14      0.58      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.12      0.57      0.20        23
                                      Imaging       0.56      0.99      0.72       133
                                  Imaging: CT       0.26      0.97      0.41        59
                                 Imaging: MRI       0.26      0.95      0.41        59
                          Imaging: Ultrasound       0.16      0.59      0.25        32
                                Manufacturing       0.35      0.93      0.51        83
        Manufacturing: Additive Manufacturing       0.21      0.79      0.33        38
           Personalized Product: Guide or Jig       0.50      0.98      0.66       120
                Personalized Product: Implant       0.52      0.99      0.68       124
                         Specification of Use       0.34      0.97      0.51        79
                Specification of Use: Disease       0.13      0.50      0.20        30
      Specification of Use: Joint Replacement       0.18      0.93      0.30        44
                              Surgical Method       0.23      0.68      0.35        40

                                    micro avg       0.33      0.91      0.49      1505
                                    macro avg       0.30      0.82      0.43      1505
                                 weighted avg       0.39      0.91      0.53      1505
                                  samples avg       0.34      0.92      0.47      1505

distilroberta-base-msmarco-v2
Pipeline(steps=[('columns',
                 ColumnTransformer(n_jobs=1,
                                   transformers=[('cited_by',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'cited_by'),
                                                 ('inventors',
                                                  CountVectorizer(analyzer=<function iden at 0x7fc9bcb59430>,
                                                                  binary=True,
                                                                  min_df=2),
                                                  'inventors'),
                                                 ('embedded_cpc',
                                                  VectorPassthrough(),
                                                  'embedded_cpc'),
                                                 ('embedding_v1',
                                                  VectorPassthrough(),
                                                  'embedding_v1'),
                                                 ('transformer_abstract',
                                                  TransformerFeatures(),
                                                  'abstract'),
                                                 ('transformer_claims',
                                                  TransformerFeatures(),
                                                  'claims')])),
                ('norm', Normalizer())])
<class 'skorch.net.NeuralNet'>[initialized](
  module_=Net(
    (layer1): Linear(in_features=6012, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (layer2): Linear(in_features=512, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=22, bias=True)
  ),
)
{'module': <class '__main__.Net'>, 'criterion': <class '__main__.LabelSmoothingCrossEntropy'>, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'max_epochs': 10000, 'batch_size': 32, 'iterator_train': <class 'torch.utils.data.dataloader.DataLoader'>, 'iterator_valid': <class 'torch.utils.data.dataloader.DataLoader'>, 'dataset': <class 'skorch.dataset.Dataset'>, 'train_split': <skorch.dataset.CVSplit object at 0x7fc7d033f460>, 'callbacks': [<skorch.callbacks.logging.ProgressBar object at 0x7fc7d0381100>, <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d0304370>, <skorch.callbacks.training.Checkpoint object at 0x7fc7d0381040>, <skorch.callbacks.training.EarlyStopping object at 0x7fc7d0381130>, <skorch.callbacks.training.LoadInitState object at 0x7fc7d03810d0>], 'predict_nonlinearity': <built-in method sigmoid of type object at 0x7fc88c1305a0>, 'warm_start': False, 'verbose': 1, 'device': 'cuda', 'module__input_features': 6012, 'module__output_features': 22, 'iterator_train__shuffle': True, 'iterator_train__num_workers': 4, 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer object at 0x7fc7c0e793d0>, 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc7c1e4fa60>, 'callbacks__train_loss__name': 'train_loss', 'callbacks__train_loss__lower_is_better': True, 'callbacks__train_loss__on_train': True, 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x7fc7c1e4f190>, 'callbacks__valid_loss__name': 'valid_loss', 'callbacks__valid_loss__lower_is_better': True, 'callbacks__valid_loss__on_train': False, 'callbacks__ProgressBar': <skorch.callbacks.logging.ProgressBar object at 0x7fc7d0381100>, 'callbacks__ProgressBar__batches_per_epoch': 'auto', 'callbacks__ProgressBar__detect_notebook': True, 'callbacks__ProgressBar__postfix_keys': ['train_loss', 'valid_loss'], 'callbacks__EpochScoring': <skorch.callbacks.scoring.EpochScoring object at 0x7fc7d0304370>, 'callbacks__EpochScoring__scoring': <function scorer at 0x7fc7d172ef70>, 'callbacks__EpochScoring__lower_is_better': False, 'callbacks__EpochScoring__on_train': False, 'callbacks__EpochScoring__name': 'f1', 'callbacks__EpochScoring__target_extractor': <function to_numpy at 0x7fc7d0383550>, 'callbacks__EpochScoring__use_caching': True, 'callbacks__Checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc7d0381040>, 'callbacks__Checkpoint__monitor': 'valid_loss_best', 'callbacks__Checkpoint__f_params': 'params.pt', 'callbacks__Checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__Checkpoint__f_criterion': 'criterion.pt', 'callbacks__Checkpoint__f_history': 'history.json', 'callbacks__Checkpoint__f_pickle': None, 'callbacks__Checkpoint__fn_prefix': '', 'callbacks__Checkpoint__dirname': 'exp2', 'callbacks__Checkpoint__event_name': 'event_cp', 'callbacks__Checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__EarlyStopping': <skorch.callbacks.training.EarlyStopping object at 0x7fc7d0381130>, 'callbacks__EarlyStopping__monitor': 'valid_loss', 'callbacks__EarlyStopping__lower_is_better': True, 'callbacks__EarlyStopping__patience': 20, 'callbacks__EarlyStopping__threshold': 0.0001, 'callbacks__EarlyStopping__threshold_mode': 'rel', 'callbacks__EarlyStopping__sink': <built-in function print>, 'callbacks__LoadInitState': <skorch.callbacks.training.LoadInitState object at 0x7fc7d03810d0>, 'callbacks__LoadInitState__checkpoint__monitor': 'valid_loss_best', 'callbacks__LoadInitState__checkpoint__f_params': 'params.pt', 'callbacks__LoadInitState__checkpoint__f_optimizer': 'optimizer.pt', 'callbacks__LoadInitState__checkpoint__f_criterion': 'criterion.pt', 'callbacks__LoadInitState__checkpoint__f_history': 'history.json', 'callbacks__LoadInitState__checkpoint__f_pickle': None, 'callbacks__LoadInitState__checkpoint__fn_prefix': '', 'callbacks__LoadInitState__checkpoint__dirname': 'exp2', 'callbacks__LoadInitState__checkpoint__event_name': 'event_cp', 'callbacks__LoadInitState__checkpoint__sink': <function noop at 0x7fc7d0383ee0>, 'callbacks__LoadInitState__checkpoint': <skorch.callbacks.training.Checkpoint object at 0x7fc7d0381040>, 'callbacks__print_log': <skorch.callbacks.logging.PrintLog object at 0x7fc7c1e4f460>, 'callbacks__print_log__keys_ignored': None, 'callbacks__print_log__sink': <built-in function print>, 'callbacks__print_log__tablefmt': 'simple', 'callbacks__print_log__floatfmt': '.4f', 'callbacks__print_log__stralign': 'right'}

In [ ]:
Using BertForPatents, dropout =0.5, bottleneck = 512, pre normalization
Hamming loss: 0.6380471380471381
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      1.00      0.52        84
           Analysis and Modeling: 3D Modeling       0.30      1.00      0.46        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.18      0.97      0.30        40
    Anatomical Target: Lower Extremity - Knee       0.34      1.00      0.50        82
                     Anatomical Target: Torso       0.15      1.00      0.27        35
             Anatomical Target: Torso - Spine       0.09      1.00      0.17        21
           Anatomical Target: Upper Extremity       0.13      0.87      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.10      0.87      0.19        23
                                      Imaging       0.55      1.00      0.71       133
                                  Imaging: CT       0.25      1.00      0.40        59
                                 Imaging: MRI       0.25      1.00      0.40        59
                          Imaging: Ultrasound       0.14      0.53      0.22        32
                                Manufacturing       0.34      1.00      0.51        83
        Manufacturing: Additive Manufacturing       0.16      0.95      0.28        38
           Personalized Product: Guide or Jig       0.49      1.00      0.66       120
                Personalized Product: Implant       0.51      1.00      0.68       124
                         Specification of Use       0.33      1.00      0.49        79
                Specification of Use: Disease       0.12      0.50      0.19        30
      Specification of Use: Joint Replacement       0.18      1.00      0.31        44
                              Surgical Method       0.20      0.72      0.32        40

                                    micro avg       0.30      0.97      0.46      1505
                                    macro avg       0.29      0.93      0.42      1505
                                 weighted avg       0.38      0.97      0.53      1505
                                  samples avg       0.31      0.97      0.45      1505

In [ ]:
Abstract + Claims + CPC Codes

Hamming loss: 0.7010849233071456
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      1.00      0.52        84
           Analysis and Modeling: 3D Modeling       0.30      1.00      0.46        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.17      1.00      0.29        40
    Anatomical Target: Lower Extremity - Knee       0.34      1.00      0.50        82
                     Anatomical Target: Torso       0.15      1.00      0.26        35
             Anatomical Target: Torso - Spine       0.09      1.00      0.16        21
           Anatomical Target: Upper Extremity       0.13      1.00      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.10      1.00      0.18        23
                                      Imaging       0.56      1.00      0.72       133
                                  Imaging: CT       0.25      1.00      0.40        59
                                 Imaging: MRI       0.25      1.00      0.40        59
                          Imaging: Ultrasound       0.14      1.00      0.24        32
                                Manufacturing       0.35      1.00      0.51        83
        Manufacturing: Additive Manufacturing       0.16      1.00      0.28        38
           Personalized Product: Guide or Jig       0.49      1.00      0.66       120
                Personalized Product: Implant       0.52      1.00      0.69       124
                         Specification of Use       0.33      1.00      0.50        79
                Specification of Use: Disease       0.13      1.00      0.22        30
      Specification of Use: Joint Replacement       0.19      1.00      0.31        44
                              Surgical Method       0.16      1.00      0.28        40

                                    micro avg       0.29      1.00      0.45      1505
                                    macro avg       0.29      1.00      0.42      1505
                                 weighted avg       0.38      1.00      0.53      1505
                                  samples avg       0.30      1.00      0.44      1505

In [ ]:
Abstract Only
{'estimator__C': 1.0, 'estimator__break_ties': False, 'estimator__cache_size': 200, 'estimator__class_weight': 'balanced', 'estimator__coef0': 0.0, 'estimator__decision_function_shape': 'ovr', 'estimator__degree': 3, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf', 'estimator__max_iter': -1, 'estimator__probability': False, 'estimator__random_state': None, 'estimator__shrinking': True, 'estimator__tol': 0.001, 'estimator__verbose': True, 'estimator': SVC(class_weight='balanced', verbose=True), 'n_jobs': -1}
Hamming loss: 0.39206883651328095
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.40      0.69      0.51        84
           Analysis and Modeling: 3D Modeling       0.33      0.70      0.45        71
                            Anatomical Target       0.73      0.86      0.79       164
           Anatomical Target: Lower Extremity       0.51      0.57      0.54       113
     Anatomical Target: Lower Extremity - Hip       0.18      0.57      0.28        40
    Anatomical Target: Lower Extremity - Knee       0.38      0.26      0.31        82
                     Anatomical Target: Torso       0.16      0.66      0.26        35
             Anatomical Target: Torso - Spine       0.07      0.14      0.09        21
           Anatomical Target: Upper Extremity       0.15      0.52      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.12      0.52      0.19        23
                                      Imaging       0.64      0.81      0.71       133
                                  Imaging: CT       0.29      0.41      0.34        59
                                 Imaging: MRI       0.31      0.15      0.20        59
                          Imaging: Ultrasound       0.14      0.12      0.13        32
                                Manufacturing       0.37      0.53      0.44        83
        Manufacturing: Additive Manufacturing       0.19      0.55      0.29        38
           Personalized Product: Guide or Jig       0.56      0.70      0.62       120
                Personalized Product: Implant       0.61      0.47      0.53       124
                         Specification of Use       0.36      0.39      0.37        79
                Specification of Use: Disease       0.16      0.13      0.15        30
      Specification of Use: Joint Replacement       0.21      0.68      0.32        44
                              Surgical Method       0.37      0.25      0.30        40

                                    micro avg       0.37      0.56      0.44      1505
                                    macro avg       0.33      0.49      0.36      1505
                                 weighted avg       0.43      0.56      0.47      1505
                                  samples avg       0.39      0.57      0.43      1505

In [ ]:
Claims Only
{'estimator__C': 1.0, 'estimator__break_ties': False, 'estimator__cache_size': 200, 'estimator__class_weight': 'balanced', 'estimator__coef0': 0.0, 'estimator__decision_function_shape': 'ovr', 'estimator__degree': 3, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf', 'estimator__max_iter': -1, 'estimator__probability': False, 'estimator__random_state': None, 'estimator__shrinking': True, 'estimator__tol': 0.001, 'estimator__verbose': True, 'estimator': SVC(class_weight='balanced', verbose=True), 'n_jobs': -1}
Hamming loss: 0.4292929292929293
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.34      0.58      0.43        84
           Analysis and Modeling: 3D Modeling       0.27      0.44      0.34        71
                            Anatomical Target       0.72      0.34      0.46       164
           Anatomical Target: Lower Extremity       0.50      0.29      0.37       113
     Anatomical Target: Lower Extremity - Hip       0.17      0.68      0.27        40
    Anatomical Target: Lower Extremity - Knee       0.40      0.23      0.29        82
                     Anatomical Target: Torso       0.14      0.34      0.20        35
             Anatomical Target: Torso - Spine       0.07      0.19      0.11        21
           Anatomical Target: Upper Extremity       0.14      0.39      0.20        31
Anatomical Target: Upper Extremity - Shoulder       0.09      0.30      0.14        23
                                      Imaging       0.60      0.78      0.68       133
                                  Imaging: CT       0.30      0.63      0.41        59
                                 Imaging: MRI       0.31      0.75      0.44        59
                          Imaging: Ultrasound       0.13      0.41      0.19        32
                                Manufacturing       0.34      0.40      0.37        83
        Manufacturing: Additive Manufacturing       0.20      0.45      0.28        38
           Personalized Product: Guide or Jig       0.55      0.59      0.57       120
                Personalized Product: Implant       0.54      0.45      0.49       124
                         Specification of Use       0.37      0.75      0.49        79
                Specification of Use: Disease       0.12      0.40      0.19        30
      Specification of Use: Joint Replacement       0.18      0.39      0.25        44
                              Surgical Method       0.44      0.20      0.28        40

                                    micro avg       0.32      0.48      0.39      1505
                                    macro avg       0.32      0.45      0.34      1505
                                 weighted avg       0.41      0.48      0.41      1505
                                  samples avg       0.35      0.50      0.38      1505
                            

In [ ]:
Abstract + Claims
{'estimator__C': 1.0, 'estimator__break_ties': False, 'estimator__cache_size': 200, 'estimator__class_weight': 'balanced', 'estimator__coef0': 0.0, 'estimator__decision_function_shape': 'ovr', 'estimator__degree': 3, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf', 'estimator__max_iter': -1, 'estimator__probability': False, 'estimator__random_state': None, 'estimator__shrinking': True, 'estimator__tol': 0.001, 'estimator__verbose': True, 'estimator': SVC(class_weight='balanced', verbose=True), 'n_jobs': -1}
Hamming loss: 0.3943135054246165
                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.39      0.70      0.50        84
           Analysis and Modeling: 3D Modeling       0.32      0.63      0.43        71
                            Anatomical Target       0.73      0.79      0.76       164
           Anatomical Target: Lower Extremity       0.47      0.35      0.40       113
     Anatomical Target: Lower Extremity - Hip       0.17      0.55      0.26        40
    Anatomical Target: Lower Extremity - Knee       0.40      0.21      0.27        82
                     Anatomical Target: Torso       0.14      0.60      0.22        35
             Anatomical Target: Torso - Spine       0.06      0.14      0.09        21
           Anatomical Target: Upper Extremity       0.16      0.48      0.24        31
Anatomical Target: Upper Extremity - Shoulder       0.14      0.52      0.22        23
                                      Imaging       0.61      0.76      0.68       133
                                  Imaging: CT       0.33      0.58      0.42        59
                                 Imaging: MRI       0.28      0.19      0.22        59
                          Imaging: Ultrasound       0.12      0.12      0.12        32
                                Manufacturing       0.35      0.45      0.39        83
        Manufacturing: Additive Manufacturing       0.22      0.55      0.32        38
           Personalized Product: Guide or Jig       0.55      0.68      0.61       120
                Personalized Product: Implant       0.60      0.47      0.52       124
                         Specification of Use       0.36      0.52      0.42        79
                Specification of Use: Disease       0.15      0.13      0.14        30
      Specification of Use: Joint Replacement       0.18      0.50      0.27        44
                              Surgical Method       0.50      0.23      0.31        40

                                    micro avg       0.36      0.52      0.43      1505
                                    macro avg       0.33      0.46      0.36      1505
                                 weighted avg       0.43      0.52      0.45      1505
                                  samples avg       0.39      0.55      0.42      1505

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

parameters = [
    {
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.7, 1.0],
    },
    {
        'classifier': [SVC()],
        'classifier__kernel': ['rbf', 'linear'],
    },
]

clf = GridSearchCV(ClassifierChain(), parameters, scoring='accuracy')
clf.fit(X_train, y_train)

print (clf.best_params_, clf.best_score_)



In [ ]:
import lightgbm as lgb
evaluate(BinaryRelevance(lgb.LGBMClassifier()))

In [ ]:
evaluate(MLkNN())

In [ ]:
evaluate(LabelPowerset(
    classifier=SVC(),
    require_dense=[False, True]
))

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import f_classif


In [ ]:
pipe = Pipeline(steps=[('transformer', transformer),
                       ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
                       #('svd', TruncatedSVD(random_state=42)),
                       #('dummy', OneVsRestClassifier(DummyClassifier()))
                       #('svc', OneVsRestClassifier(SVC(random_state=42), n_jobs=-1))
                       ('rf',  RandomForestClassifier(n_jobs=-1, random_state=42))
                       #('lr', OneVsRestClassifier(LogisticRegression(n_jobs=-1), n_jobs=-1))
                       #('cat', OneVsRestClassifier(CatBoostClassifier(verbose=True)))
                       #('knn', KNeighborsClassifier(n_jobs=-1))
                      ], 
                verbose=True,
                memory="cachedir/")

In [ ]:
training_labels[:,0]

In [ ]:
from sklearn.feature_selection import chi2
selected_features = [] 
for label in range(0, len(subset)):
    selector = SelectKBest(chi2, k='all')
    selector.fit(X, training_labels.values[:,label])
    selected_features.append(list(selector.scores_))

In [ ]:
selected_features = np.nan_to_num(np.array(selected_features))

In [ ]:
selected_features

In [ ]:
selected_features.mean(axis=0).mean()

In [ ]:
features_set = np.mean(np.nan_to_num(np.array(selected_features)), axis=0) > 0.8

In [ ]:
features_set.astype(int).sum()

In [ ]:
X.shape

In [ ]:
feature_subset = X[:,features_set]

In [ ]:
feature_subset.shape

In [ ]:
model.fit(X[:,features_set], training_labels)
predictions = model.predict(Xtest[:,features_set])
print(classification_report(testing_labels, predictions, target_names=subset))

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt import *
from skopt.space import Real, Categorical, Integer

param_grid = {
    #'rf__bootstrap': [True, False],
    'max_depth': (10, 1000), #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200, 250, 300, 350, 400, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': (1, 12),
 'min_samples_split': (2, 12),
 'n_estimators': (5, 1000)          
             }


# param_grid = {
#     'svd__n_components': Integer(64,10000),
#     'svc__estimator__C': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__gamma': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__degree': Integer(1,8),
#     'svc__estimator__kernel': Categorical(['linear', 'poly', 'rbf']),
# }

# param_grid = {
# #  'svd__n_components': np.arange(64, 5000, 100),
#  'knn__leaf_size': np.arange(1, 50, 1),
#  'knn__metric': ['minkowski', 'euclidean'],
#  'knn__n_neighbors': [2,3,4,5,6,7,8,9,10,11,12],
#  'knn__weights': ['distance', 'uniform']
# }
model = RandomForestClassifier()
search = BayesSearchCV(model, param_grid, n_iter=50, n_points=3, pre_dispatch=36, refit=True, cv=3, verbose=10, random_state=42, n_jobs=-1)

from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper

with tqdm(total=search.total_iterations) as pbar:
    def on_step(optim_result):
        print(optim_result)
        pbar.update(9)
        return False
    search.fit(X_train, y_train, callback=on_step)


In [ ]:
predictions = search.estimator.predict(X_test)

In [ ]:
f1_score(testing_labels, predictions, average="weighted")

In [ ]:
hamming_loss(testing_labels, predictions)

In [ ]:
plot_confusion_matrix(pipe, testing_set, testing_labels)

In [ ]:
training_predictions = search.best_estimator_.predict(X[:,features_set])
print(classification_report(training_labels, training_predictions, target_names=subset))

In [ ]:
from skopt import BayesSearchCV
from skopt import *
from skopt.space import Real, Categorical, Integer

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
param_grid = {
    #'rf__bootstrap': [True, False],
    'rf__max_depth': (10, 1000), #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200, 250, 300, 350, 400, None],
 #'rf__max_features': ['auto', 'sqrt'],
 'rf__min_samples_leaf': (1, 12),
 'rf__min_samples_split': (2, 12),
 'rf__n_estimators': (5, 1000)          
             }


# param_grid = {
#     'svd__n_components': Integer(64,10000),
#     'svc__estimator__C': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__gamma': Real(1e-6, 1e+6, prior='log-uniform'),
#     'svc__estimator__degree': Integer(1,8),
#     'svc__estimator__kernel': Categorical(['linear', 'poly', 'rbf']),
# }

# param_grid = {
# #  'svd__n_components': np.arange(64, 5000, 100),
#  'knn__leaf_size': np.arange(1, 50, 1),
#  'knn__metric': ['minkowski', 'euclidean'],
#  'knn__n_neighbors': [2,3,4,5,6,7,8,9,10,11,12],
#  'knn__weights': ['distance', 'uniform']
# }
search = BayesSearchCV(pipe, param_grid, n_iter=50, n_points=3, pre_dispatch=36, refit=True, cv=3, verbose=10, random_state=42, n_jobs=-1)

from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper

with tqdm(total=search.total_iterations) as pbar:
    def on_step(optim_result):
        print(optim_result)
        pbar.update(9)
        return False
    search.fit(training_set, training_labels, callback=on_step)



In [ ]:
print(search.best_estimator_.get_params())

In [ ]:
from sklearn.metrics import *
#pipe.fit(training_set, training_labels)
#predictions = pipe.predict(testing_set)
predictions = search.best_estimator_.predict(Xtest[:,features_set])
print(classification_report(testing_labels, predictions, target_names=subset))

In [ ]:
from sklearn.metrics import *
#pipe.fit(training_set, training_labels)
#predictions = pipe.predict(testing_set)
predictions = search.best_estimator_.predict(testing_set)
print(classification_report(testing_labels, predictions, target_names=all_tiers_100))

In [ ]:
from sklearn.metrics import *
#pipe.fit(training_set, training_labels)
predictions = pipe.predict(testing_set)
print(classification_report(testing_labels, predictions, target_names=all_tiers_100))

In [ ]:
predictions = predictions = search.best_estimator_.predict(training_set)
print(classification_report(training_labels, predictions, target_names=subset))


#  Guessing Baseline

                                      precision    recall  f1-score   support

                 AnalysisAndModeling       0.31      0.32      0.32        84
                    AnatomicalTarget       0.63      0.63      0.63       164
                             Imaging       0.57      0.56      0.56       133
                       Manufacturing       0.38      0.48      0.43        83
                  SpecificationofUse       0.33      0.33      0.33        79
                      SurgicalMethod       0.19      0.20      0.20        40
      AnalysisAndModeling_3DModeling       0.22      0.21      0.22        71
     AnatomicalTarget_LowerExtremity       0.45      0.43      0.44       113
              AnatomicalTarget_Torso       0.19      0.17      0.18        35
     AnatomicalTarget_UpperExtremity       0.26      0.26      0.26        31
                          Imaging_CT       0.14      0.19      0.16        59
                         Imaging_MRI       0.24      0.20      0.22        59
                  Imaging_Ultrasound       0.17      0.19      0.18        32
 Manufacturing_AdditiveManufacturing       0.24      0.24      0.24        38
       PersonalizedProduct_Guide/Jig       0.55      0.46      0.50       120
         PersonalizedProduct_Implant       0.49      0.53      0.51       124
          SpecificationofUse_Disease       0.19      0.20      0.20        30
 SpecificationofUse_JointReplacement       0.14      0.23      0.17        44
 AnatomicalTarget_LowerExtremity_Hip       0.21      0.17      0.19        40
AnatomicalTarget_LowerExtremity_Knee       0.32      0.34      0.33        82

                           micro avg       0.38      0.39      0.38      1461
                           macro avg       0.31      0.32      0.31      1461
                        weighted avg       0.38      0.39      0.39      1461
                         samples avg       0.38      0.40      0.37      1461

# RF

 {'rf__bootstrap': False,
 'rf__ccp_alpha': 0.0,
 'rf__class_weight': None,
 'rf__criterion': 'gini',
 'rf__max_depth': 150,
 'rf__max_features': 'sqrt',
 'rf__max_leaf_nodes': None,
 'rf__max_samples': None,
 'rf__min_impurity_decrease': 0.0,
 'rf__min_impurity_split': None,
 'rf__min_samples_leaf': 1,
 'rf__min_samples_split': 2,
 'rf__min_weight_fraction_leaf': 0.0,
 'rf__n_estimators': 200,
 'rf__n_jobs': None,
 'rf__oob_score': False,
 'rf__random_state': None,
 'rf__verbose': 0,
 'rf__warm_start': False}

                                      precision    recall  f1-score   support

                 AnalysisAndModeling       0.43      0.24      0.31        84
                    AnatomicalTarget       0.70      0.78      0.74       164
                             Imaging       0.60      0.59      0.60       133
                       Manufacturing       0.37      0.25      0.30        83
                  SpecificationofUse       0.42      0.32      0.36        79
                      SurgicalMethod       0.71      0.30      0.42        40
      AnalysisAndModeling_3DModeling       0.38      0.18      0.25        71
     AnatomicalTarget_LowerExtremity       0.53      0.46      0.49       113
              AnatomicalTarget_Torso       0.08      0.03      0.04        35
     AnatomicalTarget_UpperExtremity       0.11      0.03      0.05        31
                          Imaging_CT       0.18      0.10      0.13        59
                         Imaging_MRI       0.28      0.14      0.18        59
                  Imaging_Ultrasound       0.00      0.00      0.00        32
 Manufacturing_AdditiveManufacturing       0.23      0.08      0.12        38
       PersonalizedProduct_Guide/Jig       0.64      0.41      0.50       120
         PersonalizedProduct_Implant       0.59      0.74      0.66       124
          SpecificationofUse_Disease       0.06      0.03      0.04        30
 SpecificationofUse_JointReplacement       0.21      0.16      0.18        44
 AnatomicalTarget_LowerExtremity_Hip       0.26      0.12      0.17        40
AnatomicalTarget_LowerExtremity_Knee       0.47      0.33      0.39        82

                           micro avg       0.50      0.38      0.43      1461
                           macro avg       0.36      0.26      0.30      1461
                        weighted avg       0.45      0.38      0.40      1461
                         samples avg       0.53      0.41      0.43      1461



# KNN

 'knn__algorithm': 'auto',
 'knn__leaf_size': 30,
 'knn__metric': 'euclidean',
 'knn__metric_params': None,
 'knn__n_jobs': None,
 'knn__n_neighbors': 2,
 'knn__p': 2,
 'knn__weights': 'distance'
                                      precision    recall  f1-score   support

                 AnalysisAndModeling       0.38      0.43      0.40        84
                    AnatomicalTarget       0.71      0.60      0.65       164
                             Imaging       0.63      0.62      0.62       133
                       Manufacturing       0.34      0.37      0.36        83
                  SpecificationofUse       0.34      0.47      0.39        79
                      SurgicalMethod       0.39      0.30      0.34        40
      AnalysisAndModeling_3DModeling       0.35      0.38      0.36        71
     AnatomicalTarget_LowerExtremity       0.60      0.50      0.54       113
              AnatomicalTarget_Torso       0.24      0.11      0.15        35
     AnatomicalTarget_UpperExtremity       0.16      0.16      0.16        31
                          Imaging_CT       0.24      0.32      0.27        59
                         Imaging_MRI       0.24      0.32      0.28        59
                  Imaging_Ultrasound       0.16      0.28      0.21        32
 Manufacturing_AdditiveManufacturing       0.18      0.13      0.15        38
       PersonalizedProduct_Guide/Jig       0.59      0.39      0.47       120
         PersonalizedProduct_Implant       0.56      0.76      0.64       124
          SpecificationofUse_Disease       0.19      0.33      0.24        30
 SpecificationofUse_JointReplacement       0.12      0.14      0.13        44
 AnatomicalTarget_LowerExtremity_Hip       0.33      0.30      0.32        40
AnatomicalTarget_LowerExtremity_Knee       0.48      0.39      0.43        82

                           micro avg       0.43      0.44      0.43      1461
                           macro avg       0.36      0.37      0.36      1461
                        weighted avg       0.45      0.44      0.44      1461
                         samples avg       0.45      0.46      0.41      1461

# KNN + SVD

 precision    recall  f1-score   support

                 AnalysisAndModeling       0.38      0.43      0.40        84
                    AnatomicalTarget       0.71      0.60      0.65       164
                             Imaging       0.63      0.62      0.62       133
                       Manufacturing       0.34      0.37      0.36        83
                  SpecificationofUse       0.34      0.47      0.39        79
                      SurgicalMethod       0.39      0.30      0.34        40
      AnalysisAndModeling_3DModeling       0.35      0.38      0.36        71
     AnatomicalTarget_LowerExtremity       0.60      0.50      0.54       113
              AnatomicalTarget_Torso       0.24      0.11      0.15        35
     AnatomicalTarget_UpperExtremity       0.16      0.16      0.16        31
                          Imaging_CT       0.24      0.32      0.27        59
                         Imaging_MRI       0.24      0.32      0.28        59
                  Imaging_Ultrasound       0.16      0.28      0.21        32
 Manufacturing_AdditiveManufacturing       0.18      0.13      0.15        38
       PersonalizedProduct_Guide/Jig       0.59      0.39      0.47       120
         PersonalizedProduct_Implant       0.56      0.76      0.64       124
          SpecificationofUse_Disease       0.19      0.33      0.24        30
 SpecificationofUse_JointReplacement       0.12      0.14      0.13        44
 AnatomicalTarget_LowerExtremity_Hip       0.33      0.30      0.32        40
AnatomicalTarget_LowerExtremity_Knee       0.48      0.39      0.43        82

                           micro avg       0.43      0.44      0.43      1461
                           macro avg       0.36      0.37      0.36      1461
                        weighted avg       0.45      0.44      0.44      1461
                         samples avg       0.45      0.46      0.41      1461

'svd__algorithm': 'randomized',
 'svd__n_components': 2564,
 'svd__n_iter': 5,
 'svd__random_state': 42,
 'svd__tol': 0.0,
 'knn__algorithm': 'auto',
 'knn__leaf_size': 30,
 'knn__metric': 'euclidean',
 'knn__metric_params': None,
 'knn__n_jobs': None,
 'knn__n_neighbors': 2,
 'knn__p': 2,
 'knn__weights': 'distance'
